In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas_ta.volatility import atr
from pandas_ta.trend import adx
import matplotlib.pyplot as plt

import plotly.graph_objects as go

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import resample,technical_summary


c:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app\backtest


In [2]:
#klines_file  = './backtest/klines/0m01/Alts_AAVEUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
#klines_file  = './backtest/klines/0m01/Alts_GMXUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
klines_file  = './backtest/klines/0m01/Alts_GALAUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
#klines_file  = './backtest/klines/0m01/Alts_AVAXUSDT_0m01_2024-05-28_2025-05-28.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)

print('Total de velas - Iniciales',len(df))
df.reset_index(inplace=True, drop=True)
df = resample(df,periods=60*24)
print('Total de velas - Ajustadas',len(df))

#df15['atr'] = atr(df15['high'],df15['low'],df15['close'],length=14)
#df15['atr_ma'] = df15['atr'].rolling(window=14).mean()
#df15['atr_rel'] = df15['atr']/df15['atr_ma'] - 1
#df15['vol_ma'] = df15['volume'].rolling(window=100).mean()
#df15['vol_rel'] = df15['volume']/df15['vol_ma'] - 1  
#df15 = supertrend(df15)
#df15 = zigzag(df15)
#
#df15.drop(columns=['vol_ma','volume','atr','atr_ma'],inplace=True)


Total de velas - Iniciales 829440
Total de velas - Ajustadas 576


In [5]:
df['sentiment_ratio'] = 0
df['ma_sentiment_ratio'] = 0
df['osc_sentiment_ratio'] = 0

for i in df.index:
    if i>210:
        brief = technical_summary(df[i-205:i])
        df.at[i,'sentiment_ratio'] = brief['sentiment_ratio']
        df.at[i,'ma_sentiment_ratio'] = brief['ma_sentiment_ratio']
        df.at[i,'osc_sentiment_ratio'] = brief['osc_sentiment_ratio']




In [6]:
df['sr_ma'] = df['sentiment_ratio'].rolling(window=5).mean()

In [7]:
df_to_show = df

df_to_show['sl'] = None
df_to_show['tp'] = None
df_to_show['pnl'] = None
df_to_show['price'] = df_to_show['close']

indicators = [
     #{'col': 'sl','name': 'SL','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'tp','name': 'TP','color': 'green','row': 1, 'mode':'markers',},
        ]

indicators_out = [
     {'col': 'sentiment_ratio','color': 'purple','row': 1,   'mode':'bars',},
     {'col': 'sr_ma','color': 'white','row': 1,   'mode':'lines',},
     #{'col': 'ma_sentiment_ratio','color': 'red','row': 1,   'mode':'bars',},
     #{'col': 'osc_sentiment_ratio','color': 'blue','row': 1, 'mode':'bars',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       #{'df':df,'col':'sentiment_ratio','color': 'yellow','symbol': 'triangle-up' }, 
       #{'df':df,'col':'ma_sentiment_ratio','color': 'blue','symbol': 'triangle-down' }, 
       #{'df':df,'col':'osc_sentiment_ratio','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,show_pnl=False,show_volume=False,indicators=indicators,indicators_out=indicators_out,events=events,) 

fig.show()